# Implementing Logistic Regression Classifier Using Stochastic Gradient Ascent

In [1]:
import graphlab

##Loading Amazon product review dataset:-

In [3]:
products = graphlab.SFrame('amazon_baby_subset.gl/')

In [4]:
important_words = ["baby", "one", "great", "love", "use", "would", "like", "easy", "little", "seat", "old", "well", "get", 
                   "also", "really", "son", "time", "bought", "product", "good", "daughter", "much", "loves", "stroller", 
                   "put", "months", "car", "still", "back", "used", "recommend", "first", "even", "perfect", "nice", "bag", 
                   "two", "using", "got", "fit", "around", "diaper", "enough", "month", "price", "go", "could", "soft", "since", 
                   "buy", "room", "works", "made", "child", "keep", "size", "small", "need", "year", "big", "make", "take", "easily", 
                   "think", "crib", "clean", "way", "quality", "thing", "better", "without", "set", "new", "every", "cute", "best", 
                   "bottles", "work", "purchased", "right", "lot", "side", "happy", "comfortable", "toy", "able", "kids", "bit", 
                   "night", "long", "fits", "see", "us", "another", "play", "day", "money", "monitor", "tried", "thought", "never", 
                   "item", "hard", "plastic", "however", "disappointed", "reviews", "something", "going", "pump", "bottle", "cup", 
                   "waste", "return", "amazon", "different", "top", "want", "problem", "know", "water", "try", "received", "sure", 
                   "times", "chair", "find", "hold", "gate", "open", "bottom", "away", "actually", "cheap", "worked", "getting", 
                   "ordered", "came", "milk", "bad", "part", "worth", "found", "cover", "many", "design", "looking", "weeks", "say", 
                   "wanted", "look", "place", "purchase", "looks", "second", "piece", "box", "pretty", "trying", "difficult", 
                   "together", "though", "give", "started", "anything", "last", "company", "come", "returned", "maybe", "took",
                   "broke", "makes", "stay", "instead", "idea", "head", "said", "less", "went", "working", "high", "unit", "seems",
                   "picture", "completely", "wish", "buying", "babies", "won", "tub", "almost", "either"]


# Removing punctuation
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)

# Splitting out the words into individual columns
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [5]:
products

name,review,rating,sentiment,review_clean,baby
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1,All of my kids have criednonstop when I tried to ...,0
Nature's Lullabies SecondYear Sticker Calendar ...,We wanted to getsomething to keep track ...,5.0,1,We wanted to getsomething to keep track ...,0
Nature's Lullabies SecondYear Sticker Calendar ...,My daughter had her 1stbaby over a year ago. ...,5.0,1,My daughter had her 1stbaby over a year ago She ...,1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,1,One of babys first andfavorite books and it is ...,0
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,Very cute interactivebook! My son loves this ...,5.0,1,Very cute interactivebook My son loves this ...,0
Our Baby Girl Memory Book,"Beautiful book, I love itto record cherished t ...",5.0,1,Beautiful book I love itto record cherished t ...,0
Hunnt&reg; FallingFlowers and Birds Kids ...,"Try this out for a springproject !Easy ,fun and ...",5.0,1,Try this out for a springproject Easy fun and ...,0
Blessed By Pope BenedictXVI Divine Mercy Full ...,very nice Divine MercyPendant of Jesus now on ...,5.0,1,very nice Divine MercyPendant of Jesus now on ...,0
Cloth Diaper PinsStainless Steel ...,We bought the pins as my6 year old Autistic son ...,4.0,1,We bought the pins as my6 year old Autistic son ...,0
Cloth Diaper PinsStainless Steel ...,It has been many yearssince we needed diaper ...,5.0,1,It has been many yearssince we needed diaper ...,0


In [6]:
train_data, validation_data = products.random_split(.9, seed=1)

print 'Training set  : %d data points' % len(train_data)
print 'Validation set: %d data points' % len(validation_data)

Training set  : 47780 data points
Validation set: 5292 data points


##Converting SFrame to NumPy array:-

In [7]:
import numpy as np

def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [8]:
feature_matrix_train, sentiment_train = get_numpy_data(train_data, important_words, 'sentiment')
feature_matrix_valid, sentiment_valid = get_numpy_data(validation_data, important_words, 'sentiment') 

##Building on logistic regression:-

In [9]:
def predict_probability(feature_matrix, coefficients):
    # Taking dot product of feature_matrix and coefficients  
    score = np.dot(feature_matrix, coefficients)

    predictions = 1. / (1.+np.exp(-score))    
    return predictions

##Derivative of log likelihood with respect to a single coefficient:-

In [10]:
def feature_derivative(errors, feature):   
    # Computing the dot product of errors and feature
    derivative = np.dot(errors,feature)

    return derivative

In [11]:
def compute_avg_log_likelihood(feature_matrix, sentiment, coefficients):
    
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)/len(feature_matrix)
    
    return lp

##Implementing stochastic gradient ascent:-

In [13]:
from math import sqrt
def logistic_regression_SG(feature_matrix, sentiment, initial_coefficients, step_size, batch_size, max_iter):
    log_likelihood_all = []
    
    # making sure it's a numpy array
    coefficients = np.array(initial_coefficients)
    np.random.seed(seed=1)
    # Shuffle the data before starting
    permutation = np.random.permutation(len(feature_matrix))
    feature_matrix = feature_matrix[permutation,:]
    sentiment = sentiment[permutation]
    
    i = 0 # index of current batch
    # Doing a linear scan over data
    for itr in xrange(max_iter):
        # Predicting predictions using your predict_probability() function
        predictions = predict_probability(feature_matrix[i:i+batch_size,:], coefficients)
        
        # Computing indicator value for (y_i = +1)
        indicator = (sentiment[i:i+batch_size]==+1)
        
        # Computing the errors as indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # looping over each coefficient
            derivative = feature_derivative(errors, feature_matrix[i:i+batch_size,j])
            
            # computing the product of the step size, the derivative, and the **normalization constant** (1./batch_size)
            coefficients[j] = (coefficients[j]+(step_size*(derivative/batch_size)))
        
        # Checking whether log likelihood is increasing
        lp = compute_avg_log_likelihood(feature_matrix[i:i+batch_size,:], sentiment[i:i+batch_size],
                                        coefficients)
        log_likelihood_all.append(lp)
        if itr <= 15 or (itr <= 1000 and itr % 100 == 0) or (itr <= 10000 and itr % 1000 == 0) \
         or itr % 10000 == 0 or itr == max_iter-1:
            data_size = len(feature_matrix)
            print 'Iteration %*d: Average log likelihood (of data points in batch [%0*d:%0*d]) = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, \
                 int(np.ceil(np.log10(data_size))), i, \
                 int(np.ceil(np.log10(data_size))), i+batch_size, lp)
        
        # if we made a complete pass over data, shuffle and restart
        i += batch_size
        if i+batch_size > len(feature_matrix):
            permutation = np.random.permutation(len(feature_matrix))
            feature_matrix = feature_matrix[permutation,:]
            sentiment = sentiment[permutation]
            i = 0
                
    # We return the list of log likelihoods for plotting purposes.
    return coefficients, log_likelihood_all

##Running gradient ascent using the stochastic gradient ascent implementation:-

##Now **running stochastic gradient ascent** over the **feature_matrix_train** for 10 iterations using:
* `initial_coefficients = np.zeros(194)`
* `step_size = 5e-1`
* `batch_size = 1`
* `max_iter = 10`

In [15]:
coefficients, log_likelihood = logistic_regression_SG(feature_matrix_train, sentiment_train,
                                        initial_coefficients=np.zeros(194),
                                        step_size=5e-1, batch_size=1, max_iter=10)

Iteration 0: Average log likelihood (of data points in batch [00000:00001]) = -0.25192908
Iteration 1: Average log likelihood (of data points in batch [00001:00002]) = -0.00000001
Iteration 2: Average log likelihood (of data points in batch [00002:00003]) = -0.12692771
Iteration 3: Average log likelihood (of data points in batch [00003:00004]) = -0.02969101
Iteration 4: Average log likelihood (of data points in batch [00004:00005]) = -0.02668819
Iteration 5: Average log likelihood (of data points in batch [00005:00006]) = -0.04332901
Iteration 6: Average log likelihood (of data points in batch [00006:00007]) = -0.02368802
Iteration 7: Average log likelihood (of data points in batch [00007:00008]) = -0.12686897
Iteration 8: Average log likelihood (of data points in batch [00008:00009]) = -0.04468879
Iteration 9: Average log likelihood (of data points in batch [00009:00010]) = -0.00000124


In [16]:
coefficients

array([ 0.26845909,  0.05510662, -0.78232359,  0.24929641,  0.1213813 ,
       -0.13194118, -0.42110769,  0.23944013,  0.52334226,  0.30746343,
        1.46697311,  0.15734639,  0.24112255, -0.22849175, -0.48095714,
        0.        ,  0.05984944, -0.41942527, -0.48095714,  0.10654088,
        0.        ,  0.06153186, -0.41942527,  0.43843464,  0.        ,
        0.21719583,  0.        ,  0.84326475,  0.28108825,  0.28108825,
        0.        ,  0.        ,  0.24611428, -0.19986888,  0.15734639,
        0.        ,  0.        , -0.48095714,  0.12623269,  0.        ,
        0.28108825,  0.07542718,  0.        , -0.42110769,  0.15734639,
       -0.48095714,  0.24611428, -0.48095714,  0.        ,  0.        ,
        0.06153186,  0.28108825,  0.        ,  0.        ,  0.        ,
        0.05984944,  0.5932902 ,  0.5621765 , -0.48095714,  0.        ,
        0.05984944,  0.05984944,  0.31220195,  0.11805882,  0.        ,
        0.15085436,  0.24611428,  0.        ,  0.        ,  0.  

##Now running **batch gradient ascent** over the **feature_matrix_train** for 200 iterations using:
* `initial_coefficients = np.zeros(194)`
* `step_size = 5e-1`
* `batch_size = len(feature_matrix_train)`
* `max_iter = 200`

In [17]:
coefficients_batch, log_likelihood_batch = logistic_regression_SG(feature_matrix_train, sentiment_train,
                                        initial_coefficients=np.zeros(194),
                                        step_size=5e-1, batch_size=len(feature_matrix_train), max_iter=200)

Iteration   0: Average log likelihood (of data points in batch [00000:47780]) = -0.68308119
Iteration   1: Average log likelihood (of data points in batch [00000:47780]) = -0.67394599
Iteration   2: Average log likelihood (of data points in batch [00000:47780]) = -0.66555129
Iteration   3: Average log likelihood (of data points in batch [00000:47780]) = -0.65779626
Iteration   4: Average log likelihood (of data points in batch [00000:47780]) = -0.65060701
Iteration   5: Average log likelihood (of data points in batch [00000:47780]) = -0.64392241
Iteration   6: Average log likelihood (of data points in batch [00000:47780]) = -0.63769009
Iteration   7: Average log likelihood (of data points in batch [00000:47780]) = -0.63186462
Iteration   8: Average log likelihood (of data points in batch [00000:47780]) = -0.62640636
Iteration   9: Average log likelihood (of data points in batch [00000:47780]) = -0.62128063
Iteration  10: Average log likelihood (of data points in batch [00000:47780]) = 

In [19]:
coefficients_batch

array([ -8.37459073e-02,   9.08617833e-02,   1.15758444e-02,
         7.95760232e-01,   1.03258253e+00,  -7.04301057e-04,
        -2.88517277e-01,  -6.02975644e-03,   9.66628207e-01,
         5.29720107e-01,  -1.01819059e-01,   2.03016450e-01,
         4.58432979e-01,  -1.83018437e-01,   1.61839178e-01,
        -1.41729267e-02,   1.40335271e-01,  -7.92282621e-02,
        -1.31915781e-01,  -2.59626178e-01,   1.46436905e-01,
         2.68132028e-01,  -1.28677605e-02,   1.02861545e+00,
        -2.95690813e-02,  -3.08451941e-03,  -8.95332632e-02,
         1.93766592e-01,   1.93346623e-01,  -2.64047921e-01,
         9.88729037e-02,   3.83478049e-01,  -7.80390284e-02,
        -3.81376652e-01,   8.14508917e-01,   4.32871857e-01,
        -1.64200145e-02,  -1.13674444e-01,   3.16902198e-02,
        -8.66264301e-02,  -1.46244114e-01,   1.11772527e-01,
         4.20327135e-02,   2.07281321e-02,  -1.80097413e-01,
         2.39974805e-01,   6.61739071e-02,  -1.73116476e-01,
         3.56059683e-01,